In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
import sys

In [3]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
from posarmctools.readdata import *

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *
from posarutils.other.PosarMCParameters import *

In [6]:
data_date = "2018_05_18_14_45_06"
data_dir = "/home/pleroy/DATA/PoSAR-v2_PIMA_TEST-3/2018_05_18/" + data_date

In [7]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters_v2( params_filename )

In [8]:
buffersPerFile = params.buffersPerFile
samplesPerRamp = params.samplesPerRamp
rampsPerFile = params.rampsPerFile
samplesPerFile = samplesPerRamp * rampsPerFile

print( "buffersPerFile = {}, samplesPerRamp = {}, rampsPerFile = {}".format(
    buffersPerFile, samplesPerRamp, rampsPerFile )
     )

# selection in record
firstFile = 18
nbFiles = 146
lastFile = firstFile + nbFiles - 1
firstBuffer = firstFile * buffersPerFile

buffersPerFile = 2, samplesPerRamp = 12000, rampsPerFile = 750


# Read the data

In [9]:
scalingFactor = 2 / 65535
offset = -32768

In [10]:
timeSerie_A = np.zeros( samplesPerFile )
adc_A = np.zeros( (nbFiles, samplesPerFile) )

## Read all files

In [11]:
lastBuffer = (nbFiles-1) * buffersPerFile + firstBuffer
for k in range( nbFiles ):
    
    # read the data
    nb = str( int( k * buffersPerFile + firstBuffer) )
    filename = data_dir + "/record" + nb + ".bin"
    readFileADLINKCh0( filename, samplesPerFile, timeSerie_A )
    
    print("k = {}, block = {} / {}".format(k, nb, lastBuffer))
    
    adc_A[ k, : ] = (timeSerie_A + offset) * scalingFactor

k = 0, block = 36 / 326
k = 1, block = 38 / 326
k = 2, block = 40 / 326
k = 3, block = 42 / 326
k = 4, block = 44 / 326
k = 5, block = 46 / 326
k = 6, block = 48 / 326
k = 7, block = 50 / 326
k = 8, block = 52 / 326
k = 9, block = 54 / 326
k = 10, block = 56 / 326
k = 11, block = 58 / 326
k = 12, block = 60 / 326
k = 13, block = 62 / 326
k = 14, block = 64 / 326
k = 15, block = 66 / 326
k = 16, block = 68 / 326
k = 17, block = 70 / 326
k = 18, block = 72 / 326
k = 19, block = 74 / 326
k = 20, block = 76 / 326
k = 21, block = 78 / 326
k = 22, block = 80 / 326
k = 23, block = 82 / 326
k = 24, block = 84 / 326
k = 25, block = 86 / 326
k = 26, block = 88 / 326
k = 27, block = 90 / 326
k = 28, block = 92 / 326
k = 29, block = 94 / 326
k = 30, block = 96 / 326
k = 31, block = 98 / 326
k = 32, block = 100 / 326
k = 33, block = 102 / 326
k = 34, block = 104 / 326
k = 35, block = 106 / 326
k = 36, block = 108 / 326
k = 37, block = 110 / 326
k = 38, block = 112 / 326
k = 39, block = 114 / 326
k 

In [12]:
A_reshaped = adc_A.reshape(nbFiles * rampsPerFile, samplesPerRamp)

**Check that the first ramp is an up ramp**

In [13]:
plt.figure()
plt.plot( A_reshaped[0, :] )
plt.title("first acquisition\n" + data_date)
plt.grid()

**Set the shifted flag properly**

In [14]:
rampDownFirst = 1

## Build RD

In [15]:
withHanning = 1
RDc = build_rd_from_data_rampDown( params, A_reshaped, rampDownFirst, withHanning )

shape of the samples matrix = (109500, 12000)
with Hanning window
ramp down first in the data files


In [16]:
coupling_RD = np.average(RDc, 0)
if withHanning:
    np.save( data_dir + "/coupling_RD_files_{}_{}_rampDown_hanning".format(firstFile, lastFile), coupling_RD )
else:
    np.save( data_dir + "/coupling_RD_files_{}_{}_rampDown".format(firstFile, lastFile), coupling_RD )

In [17]:
plt.figure()
plt.plot(np.abs(coupling_RD), 'k', label='coupling')
plt.legend()
plt.grid()

In [18]:
if withHanning:
    np.save(data_dir + '/RD_files_{}_{}_rampDown_hanning'.format(firstFile, lastFile), RDc) 
else:
    np.save(data_dir + '/RD_files_{}_{}_rampDown'.format(firstFile, lastFile), RDc)